In [1]:
import requests
from requests_oauthlib import OAuth1

In [2]:
import urllib
import cnfg
from pprint import pprint
config = cnfg.load(".twitter_config")

In [3]:
oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

In [4]:
import json
from pymongo import MongoClient

client = MongoClient()
db = client.fletcher
grammy_col = db.grammy

In [5]:
main_keys = ['created_at', 'id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_count', 'text']
user_keys = ['created_at', 'description', 'followers_count', 'friends_count', 'id', 'location', 'name', 'screen_name']
def insertTweet(collection, tweet):
    data = {}
    for key in main_keys:
        data[key] = tweet[key]
    for key in user_keys:
        data["user_" + key] = tweet['user'][key]
    collection.insert_one(data)
        

In [89]:
parameters = {"q": "%23grammys+OR%23grammy", "count":100, "until": "2017-02-13", "lang":"en", "max_id": "830816541583278080"}
response = requests.get("https://api.twitter.com/1.1/search/tweets.json",
                        params = parameters,
                        auth=oauth)

In [91]:
n_params = response.json()["search_metadata"]['next_results']
print(n_params)

?max_id=830816492459540479&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1


In [92]:
twitter_response = response.json()['statuses']
for tweet in twitter_response:
    insertTweet(grammy_col, tweet)

In [35]:
import time

In [93]:
for i in range(100):
    for i in range(50):
        response = requests.get("https://api.twitter.com/1.1/search/tweets.json" + n_params, auth=oauth)
        n_params = response.json()["search_metadata"]["next_results"]
        twitter_response = response.json()['statuses']
        for tweet in twitter_response:
            insertTweet(grammy_col, tweet)
        print(n_params)
    time.sleep(15*60) # sleep for 15 minutes

?max_id=830816444669636607&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830816386414899199&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830816314511986687&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830816237680680961&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830816146752409600&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830816072290938879&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830815982469910527&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830815900685066239&q=%2523grammys%2BOR%2523grammy%20until%3A2017-02-13&lang=en&count=100&include_entities=1
?max_id=830815818279694336&q=%2523grammys%2BOR%2523grammy%20until%3A2017

KeyError: 'next_results'

In [94]:
twitter_response = response.json()['statuses']

In [95]:
len(twitter_response)

98

In [96]:
for tweet in twitter_response:
    insertTweet(grammy_col, tweet)

In [98]:
twitter_response[-1]['text']

'RT @ArianatorIsland: THIS WOMAN DESERVES A GRAMMY TONIGHT 🗣🗣🗣🗣🗣 #GRAMMYs https://t.co/sF5raGHjIG'

In [99]:
grammy_col.count()

203336

In [6]:
tweets = []
for tweet in grammy_col.find():
    tweets.append(tweet['text'])

In [12]:
tweets[-100:]

['RT @OnlyHipHopFacts: Tonight. #Grammys https://t.co/b4jMEQX5Mf',
 'A star-studded #RocNationBrunch ahead of the #Grammys https://t.co/Bqnz8QFOlv',
 'RT @eionclarke: This album is a winner #GRAMMYs https://t.co/UtkmYnvBoH',
 'RT @BieberFanAlan: This is stan twitter tonight #GRAMMYs https://t.co/iKNfO5uAIQ',
 'RT @JustinItalyHelp: Retweet if you think that @justinbieber deserves to win the #GRAMMYs',
 'RT @RecordingAcad: The final touches are being added to the red carpet! ♥ Who is ready for the #GRAMMYs tonight? https://t.co/zFBYoWWtOD',
 'RT @ArmaanMalik22: Best of luck @justinbieber for the #GRAMMYs | Hope #Purpose &amp; you as an artiste get the recognition that is deserved! 🙌🏼❤',
 'RT @BieberFanAlan: This album is a winner #GRAMMYs\xa0 #Beliebers #BestFanArmy #iHeartsAwards https://t.co/sPh1uhINRq',
 "RT @ItsFajarBaig: Remember when this pic was Justin's icon? #Beliebers #BestFanArmy #iHeartAwards #GRAMMYs https://t.co/OrPRqMLbMk",
 'RT @Bey_Legion: #Throwback: Tina Turner &amp; B